<a href="https://colab.research.google.com/github/saided/pandas_using_python/blob/master/grp_create.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import requests
import xlrd
import openpyxl
from collections import ChainMap
from requests import Timeout

In [ ]:
payload={'page':2,'count':25}
r = requests.get('https://httpbin.org/get', params=payload)
#print(r.text)
#print(r.url)
print(r.headers['Content-Type'])

application/json


In [ ]:
payload={'username':'sai','password':'test123'}
r = requests.post('https://httpbin.org/post', data=payload)
#print(r.text)
r_dict = r.json()
#print(r.json()) # get response in a dict format
print(r_dict['form']['password'])



test123


In [ ]:
#basic-auth/{user}/{pwd}
r = requests.get('https://httpbin.org/basic-auth/sai/test123',auth=('sai','test123'))
print(r.text)


{
  "authenticated": true, 
  "user": "sai"
}



In [23]:
 url = "https://api.cloud.jiosurveillance.com/auth/oauth/token"
 r = requests.get(url)
print(r.headers)
#print(r.json())

{'Content-Type': 'text/html; charset=UTF-8', 'Server': 'IVIDEON', 'access-control-allow-headers': 'Access-Control-Allow-Headers, Origin, Accept, X-Requested-With, Content-Type, Access-Control-Request-Method, Access-Control-Request-Headers, Authorization', 'access-control-allow-origin': '*', 'access-control-allow-methods': 'GET, POST, OPTIONS', 'x-service': 'auth-server', 'x-node': 'azprivvsapp08, azdeivvsfro02', 'Date': 'Thu, 23 Jul 2020 15:46:19 GMT', 'Content-Length': '87'}


In [90]:
def root_group():
    url = "https://api.cloud.jiosurveillance.com/auth/oauth/token"
    username = input("Enter account ID")
    #print(username)
    password = input("Enter account password")
    #print(password)

    payload = 'client_secret=8b078e3496893ae3b9d7ccb83a6befdb&client_id=firmware-upgrade-job&grant_type=password&username={}&password={}'.format(
    username, password)
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
            }
    try:
        response = requests.request("POST", url, headers=headers, data=payload, timeout=6)
        #print(response.text.encode('utf8'))
        #print(response.json())
    except: Timeout

    r = response.json()
    access_token = r['access_token']
    token = str(access_token)
    #print(token)
    own_id = r['owner_id']
    own_id = str(own_id)
    #print(own_id)

    url = "https://api.cloud.jiosurveillance.com/folders?op=FIND"

    payload = "{\n\t\"user\" : \"%s\"\n}" % (own_id)
    headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % (token)
    }
    #print(payload)
    response = requests.request("POST", url, headers=headers, data=payload, timeout=6)
    #print(response.text.encode('utf8')) # it gives total info about groups present in that  account
    #print(response.url)

    own_response = response.json()
    b = own_response['result']
    #print(b)
    c = b['items']
    #print(c[0])
    #print(len(c))
    #print('Total no.of existing groups in the account :',len(c)-1)
    mast_grp_id = c[0]['id']
    #print(mast_grp_id,type(mast_grp_id))
    if len(c)==0:
      print('something went wrong')
    return(mast_grp_id,own_id,token)

print(root_group())  

Enter account IDsaitaddi199733@gmail.com
Enter account passwordJioril123
('801-K0FbVNA2tX', '801000010157', '801-Ufd44d24e-4a68-4b57-b478-10ded49e4e0e')


In [99]:
def create_state(cellvalue,grp_id,token):
    url = "https://api.cloud.jiosurveillance.com/folders?op=CREATE"

    payload = '{\n\t\"parent_folder\" : \"' + grp_id + '\",\n\t\"name\" : \"' + cellvalue + "\"\n}"
    print(payload)
    headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer %s' % token
    }

    response = requests.request("POST", url, headers=headers, data=payload, timeout=6)
    print(response.text.encode('utf8'))

    return 1


In [ ]:
def get_folder_id(cell_content, token, own_id, master_id):
    url = "https://api.cloud.jiosurveillance.com/folders?op=FIND"

    payload = "{\n\t\"user\" : \"" + own_id + "\"\n}"
    print("Payload for get_folder_id is" + payload)
    headers = {
      'Content-Type': 'application/json',
      'Authorization': 'Bearer %s' % token
    }

    response = requests.request("POST", url, headers=headers, data = payload, timeout=6)

    print(response.text.encode('utf8'))
    get_folder_id_response = response.json()

In [93]:
is_root_grp = list(root_group())
print(len(is_root_grp))
mast_grp_id = is_root_grp[0]
own_id = is_root_grp[1]
token = is_root_grp[2]
#print(token,type(token))

Enter account IDsaitaddi199733@gmail.com
Enter account passwordJioril123
3


In [100]:
North=0
theFile = openpyxl.load_workbook('/content/grp_test.xlsx')
allSheetNames = theFile.sheetnames
#print(allSheetNames)
print("All sheet names {} " .format(allSheetNames))
for sh in xlrd.open_workbook('/content/grp_test.xlsx').sheets():
  print('Total no of rows',sh.nrows)
  print('Total no of columns',sh.ncols)
  dict={}
  for row in range(1, sh.nrows):
    print(row)
    cellcontent=[]
    for col in range(sh.ncols):
      mycell = sh.cell(row,col).value
      #print(mycell)
      #cellcontent.append(mycell)
      #dict[row]= cellcontent
    #print(cellcontent)
    #print(dict)
      if (len(is_root_grp) !=0 and col==1):
        if (mycell=='North' and North==0):
          create_state(mycell,mast_grp_id,token)
          North+=1
        else:
          print("Repeated")
          break
northList = []
  for row in range(1, sh.nrows):
    for col in range(1, 3):
      mycell = sh.cell(row,col).value
      if mycell=='North':
        statecell = sh.cell(row, col+1)
        stateCell.value = str(stateCell.value)
        if stateCell.value.lower() in [name.lower() for name in northList]:
          break
        else:
          northList.append(stateCell.value)
          print("NorthList contains following list" + str(northList))
          folder_id = get_folder_id(cell_content, token, own_id, root_grp)
          print("Received folder id for NORTH is" + folder_id[0])
          #check_state_present = check_state_created(cell_content, folder_id)
          create_folder(folder_id[0], stateCell.value, token)




All sheet names ['Sheet1'] 
Total no of rows 10
Total no of columns 9
1
{
	"parent_folder" : "801-K0FbVNA2tX",
	"name" : "North"
}
b'{"success": true, "result": {"id": "801-IDWHRr52OC", "name": "North", "subfolders": [], "cameras": [], "owner_id": "801000010157", "owner_name": "saitaddi199733@gmail.com", "root": false, "permissions": ["admin", "live", "archive", "ptz", "events", "faces_read", "faces_add", "faces_admin"], "geo": null}}'
2
Repeated
3
Repeated
4
Repeated
5
Repeated
6
Repeated
7
Repeated
8
Repeated
9
Repeated
